## Program takes data.csv file as input which contains raw data 
## Then it performs
          1.Data cleaning, 
          2.Categorical value to numerical value conversion
          3.Data Normalization
          4.Data Partition
## During Data partition, it splits data into
           1.Training data     It contains 75% of the whole dataset
           2.Validation data   It contains 15% of the whole dataset
           3.Test data         It contains 10% of the whole dataset
## Then Program use Keras to build Neural Network, train the network and evaluate the network
## Result:
            1. Taining Accuracy = 97.41%
            2. Validation Accuracy = 97.35%
            3. Test Accuracy = 97.27%
## On average Accuracy lies arround 97%

In [405]:
import pandas as pd
import numpy as np
import time
import datetime
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.core import Dropout
from keras.utils import to_categorical
from keras.regularizers import l2
from keras.models import load_model

In [406]:
#load csv file to the pandas dataframe
data = pd.read_csv('data.csv')

### Data Visualization

In [407]:
data.head(10)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.00,failed,0,GB,0.00,0.00,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.00,failed,15,US,100.00,2421.00,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.00,failed,3,US,220.00,220.00,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.00,failed,1,US,1.00,1.00,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.00,canceled,14,US,1283.00,1283.00,19500.00
5,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01,50000.0,2016-02-26 13:38:27,52375.00,successful,224,US,52375.00,52375.00,50000.00
6,1000023410,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,USD,2014-12-21,1000.0,2014-12-01 18:30:44,1205.00,successful,16,US,1205.00,1205.00,1000.00
7,1000030581,Chaser Strips. Our Strips make Shots their B*tch!,Drinks,Food,USD,2016-03-17,25000.0,2016-02-01 20:05:12,453.00,failed,40,US,453.00,453.00,25000.00
8,1000034518,SPIN - Premium Retractable In-Ear Headphones w...,Product Design,Design,USD,2014-05-29,125000.0,2014-04-24 18:14:43,8233.00,canceled,58,US,8233.00,8233.00,125000.00
9,100004195,STUDIO IN THE SKY - A Documentary Feature Film...,Documentary,Film & Video,USD,2014-08-10,65000.0,2014-07-11 21:55:48,6240.57,canceled,43,US,6240.57,6240.57,65000.00


In [408]:
# I used data.info() and data.describe to see the distribution of the data 

In [409]:
# Used groupby from pandas to see each category of the 

In [410]:
group_state = data.groupby('state')

In [411]:
len(group_state.groups)

6

In [412]:
group_main_category = data.groupby('main_category')

In [413]:
len(group_main_category.groups)

15

In [414]:
group_currency = data.groupby('currency')

In [415]:
len(group_currency.groups)

14

In [416]:
groups_country = data.groupby('country')

In [417]:
len(groups_country)

23

## Get successful and failed data from the whole dataset

In [418]:
# Filter out other Projects which are in canceled,live,suspended and undefined state
# Because from these data we can not get information whether the project is going to be 
# successful or not
success_fail_data = data.loc[(data['state']=='successful')|(data['state']=='failed')]
# Now there are 331,675 rows left

In [419]:
success_fail_data.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
5,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01,50000.0,2016-02-26 13:38:27,52375.0,successful,224,US,52375.0,52375.0,50000.00


## Filter the required columns which might have the high impact on output

In [420]:
# ID, name might not have huge impact in the success of the project
filtered_data = success_fail_data.filter(items = ['main_category','currency','state','backers',\
                                                  'country','usd_pledged_real','usd_goal_real'])

In [421]:
filtered_data.head()

,main_category,currency,state,backers,country,usd_pledged_real,usd_goal_real
0,Publishing,GBP,failed,0,GB,0.0,1533.95
1,Film & Video,USD,failed,15,US,2421.0,30000.00
2,Film & Video,USD,failed,3,US,220.0,45000.00
3,Music,USD,failed,1,US,1.0,5000.00
5,Food,USD,successful,224,US,52375.0,50000.00


## Convert string datatype to integer datatype

In [422]:
Main_category = {'Art':1,'Comics':2,'Crafts':3,'Dance':4,'Design':5,'Fashion':6,'Film & Video':7,\
                 'Food':8,'Games':9,'Journalism':10,'Music':11,'Photography':12,'Publishing':13,'Technology':14\
                ,'Theater':15}
Currency = {'AUD':1,'CAD':2,'CHF':3,'DKK':4,'EUR':5,'GBP':6,'HKD':7,'JPY':8,'MXN':9,'NOK':10,'NZD':11,'SEK':12\
           ,'SGD':13,'USD':14}
Country = {'AT':1,'AU':2,'BE':3,'CA':4,'CH':5,'DE':6,'DK':7,'ES':8,'FR':9,'GB':10,'HK':11,'IE':12,'IT':13\
          ,'JP':14,'LU':15,'MX':16,'N,0"':17,'NL':18,'NO':19,'NZ':20,'SE':21,'SG':22,'US':23}
State = {'failed':0,'successful':1}

## Normalization of the data

In [423]:
filtered_data.state = [State[item] for item in filtered_data.state]
filtered_data.main_category = [Main_category[item]/15 for item in filtered_data.main_category]
filtered_data.currency = [Currency[item]/14 for item in filtered_data.currency]
filtered_data.country = [Country[item]/23 for item in filtered_data.country]
max_backers = filtered_data.backers.max()
max_usd_pledged_real = filtered_data.usd_pledged_real.max()
max_usd_goal_real = filtered_data.usd_goal_real.max()
filtered_data.backers = [item/max_backers for item in filtered_data.backers]
filtered_data.usd_pledged_real = [item/max_usd_pledged_real for item in filtered_data.usd_pledged_real]
filtered_data.usd_goal_real = [item/max_usd_goal_real for item in filtered_data.usd_goal_real]

In [424]:
filtered_data.head()

,main_category,currency,state,backers,country,usd_pledged_real,usd_goal_real
0,0.866667,0.428571,0,0.000000,0.434783,0.000000e+00,0.000009
1,0.466667,1.000000,0,0.000068,1.000000,1.190325e-04,0.000180
2,0.466667,1.000000,0,0.000014,1.000000,1.081666e-05,0.000270
3,0.733333,1.000000,0,0.000005,1.000000,4.916666e-08,0.000030
5,0.533333,1.000000,1,0.001021,1.000000,2.575104e-03,0.000301


In [425]:
filtered_data.describe()

,main_category,currency,state,backers,country,usd_pledged_real,usd_goal_real
count,331675.000000,331675.000000,331675.000000,331675.000000,331675.000000,331675.000000,3.316750e+05
mean,0.565561,0.862945,0.403877,0.000530,0.869340,0.000489,2.495170e-04
std,0.262098,0.277041,0.490674,0.004401,0.269297,0.004756,6.665787e-03
min,0.066667,0.071429,0.000000,0.000000,0.043478,0.000000,6.011010e-11
25%,0.400000,1.000000,0.000000,0.000009,1.000000,0.000002,1.202202e-05
50%,0.533333,1.000000,0.000000,0.000068,1.000000,0.000039,3.005505e-05
75%,0.733333,1.000000,1.000000,0.000287,1.000000,0.000227,9.016515e-05
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00


## Split failed project and successful project to balance the data set

In [426]:
#success_fail_data = data.loc[(data['state']=='successful')|(data['state']=='failed')]
success_project = filtered_data.loc[filtered_data['state']==1]
failed_project = filtered_data.loc[filtered_data['state']==0]

## Split test, validation and training data from individual failed and successful project

In [427]:
# To randomly shuffle the successful project data
success_project = success_project.sample(frac=1)
# To seperate 10% of data for the test data from successful project
test_length_success = int(len(success_project)*0.1) 
#To seperate 15% of data for the validation from succesful project
validation_length_success = int(len(success_project)*0.25)
success_test_data = success_project[:test_length_success :]
success_validation_data = success_project[test_length_success:validation_length_success :]
success_train_data = success_project[validation_length_success: :]

In [428]:
print(len(success_test_data))
print(len(success_validation_data))
print(len(success_train_data))

13395
20094
100467


In [429]:
print(len(failed_project))

197719


## This data set is unblanced, there are 133,956 successful projects and 197719 failed,
## unsuccessful, project. So I balanced the data set by using following steps

In [430]:
# To randomly shuffle and get only 67.751% of the failed project data
# Main purpose of this line is to balance the number of successful and failed project
failed_project = failed_project.sample(frac=len(success_project)/len(failed_project))
# To seperate 10% of data for the test data from failed project
print('Now new length of failed_project is=', len(failed_project))
test_length_failed = int(len(failed_project)*0.1) 
#To seperate 15% of data for the validation from failed project
validation_length_failed = int(len(failed_project)*0.25)
failed_test_data = failed_project[:test_length_failed :]
failed_validation_data = failed_project[test_length_failed:validation_length_failed :]
failed_train_data = failed_project[validation_length_failed: :]

Now new length of failed_project is= 133956


In [431]:
print(len(failed_test_data))
print(len(failed_validation_data))
print(len(failed_train_data))

13395
20094
100467


## Merge test data from the failed project to the test data from the successful project

In [432]:
final_test_data = pd.concat([success_test_data,failed_test_data],axis = 0)
# Randomly shuffle the final_test_data to fixed the successful test data and failed test data
final_test_data = final_test_data.sample(frac=1)

## Merge validation data from the failed project to the validation data from the successful project

In [433]:
final_validation_data = pd.concat([success_validation_data,failed_validation_data],axis = 0)
# Randomly shuffle the final_validaion_data to fixed the successful validaion data and failed validaion data
final_validation_data = final_validation_data.sample(frac=1)

## Merge training data from the failed project to the training data from the successful project

In [434]:
final_training_data = pd.concat([success_train_data,failed_train_data],axis = 0)
# Randomly shuffle the final_training_data to fix the successful training data and failed training data
final_training_data = final_training_data.sample(frac = 1)

## Write the final test data, final validation data and final training data to the csv file

In [435]:
final_test_data.to_csv('processed_data/final_test_data.csv')
final_validation_data.to_csv('processed_data/final_validation_data.csv')
final_training_data.to_csv('processed_data/final_training_data.csv')

## Seperate inputs from label

In [436]:
train_inputs = final_training_data.filter(items = ['main_category','currency','backers',\
                                                  'country','usd_pledged_real','usd_goal_real'])
train_labels = final_training_data.filter(items = ['state'])

In [437]:
train_inputs.head()

,main_category,currency,backers,country,usd_pledged_real,usd_goal_real
113607,0.933333,0.357143,0.000009,0.652174,1.076750e-07,6.575444e-07
363815,0.733333,1.000000,0.000492,1.000000,5.971291e-04,6.011010e-05
304484,0.866667,0.428571,0.000520,0.434783,4.168610e-04,1.364449e-04
213322,0.733333,1.000000,0.000005,1.000000,7.374999e-07,7.213212e-06
28016,0.866667,1.000000,0.000324,1.000000,1.947000e-04,5.409909e-06


In [438]:
train_labels.head()

,state
113607,0
363815,1
304484,0
213322,0
28016,1


In [439]:
test_inputs = final_test_data.filter(items = ['main_category','currency','backers',\
                                                  'country','usd_pledged_real','usd_goal_real'])
test_labels = final_test_data.filter(items = ['state'])

validation_inputs = final_validation_data.filter(items = ['main_category','currency','backers',\
                                                  'country','usd_pledged_real','usd_goal_real'])
validation_labels = final_validation_data.filter(items = ['state'])

In [440]:
test_inputs.head()

,main_category,currency,backers,country,usd_pledged_real,usd_goal_real
91947,0.533333,1.0,0.000005,1.0,0.000002,0.000150
92035,0.866667,1.0,0.000191,1.0,0.000108,0.000096
254103,0.600000,1.0,0.000036,1.0,0.000263,0.000150
313233,0.533333,1.0,0.000027,1.0,0.000015,0.000001
19180,0.866667,1.0,0.000292,1.0,0.000620,0.000102


## Convert pandas dataframe to numpy ndarray for the trainging

In [441]:
numpy_train_inputs = train_inputs.as_matrix()
numpy_train_labels = train_labels.as_matrix()
numpy_test_inputs = test_inputs.as_matrix()
numpy_test_labels = test_labels.as_matrix()
numpy_validation_inputs = validation_inputs.as_matrix()
numpy_validation_labels = validation_labels.as_matrix()

In [442]:
numpy_train_labels[:5]

array([[0],
       [1],
       [0],
       [0],
       [1]])

### Convert numpy array labels to categorical value
#### Here first or top node represent the zero if top node gets the output value near 1 that means
####  that particular project is unsuccess one. 
#### Same principle follows to the success node, buttom node.

In [443]:
train_label_categ = to_categorical(numpy_train_labels)
test_label_categ = to_categorical(numpy_test_labels)
validation_label_categ = to_categorical(numpy_validation_labels)

In [444]:
train_label_categ[:5]

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [445]:
type(numpy_train_inputs)

numpy.ndarray

In [446]:
numpy_train_inputs[0]

array([9.33333333e-01, 3.57142857e-01, 9.11651822e-06, 6.52173913e-01,
       1.07674983e-07, 6.57544395e-07])

In [447]:
numpy_train_labels.shape

(200934, 1)

# Train Model Using keras

In [448]:
model = Sequential([
    Dense(12,activation='relu',input_shape = (6,)),
    Dense(12,activation='relu'),
    Dense(2,activation='softmax')
])

In [449]:
model.compile(optimizer='adam',loss = 'categorical_crossentropy',
               metrics = ['accuracy'])

In [450]:
model.fit(numpy_train_inputs,train_label_categ,epochs = 200, batch_size=128,
            validation_data=(numpy_validation_inputs,validation_label_categ))

Train on 200934 samples, validate on 40188 samples
Epoch 1/200
200934/200934 [==============================] - 3s 15us/step - loss: 0.6906 - acc: 0.5219 - val_loss: 0.6890 - val_acc: 0.5412
Epoch 2/200
200934/200934 [==============================] - 2s 10us/step - loss: 0.6829 - acc: 0.5534 - val_loss: 0.6712 - val_acc: 0.5911
Epoch 3/200
200934/200934 [==============================] - 2s 11us/step - loss: 0.6473 - acc: 0.6349 - val_loss: 0.6207 - val_acc: 0.6562
Epoch 4/200
200934/200934 [==============================] - 2s 11us/step - loss: 0.5909 - acc: 0.6999 - val_loss: 0.5592 - val_acc: 0.7588
Epoch 5/200
200934/200934 [==============================] - 2s 11us/step - loss: 0.5385 - acc: 0.7474 - val_loss: 0.5113 - val_acc: 0.7587
Epoch 6/200
200934/200934 [==============================] - 2s 12us/step - loss: 0.4952 - acc: 0.7839 - val_loss: 0.4734 - val_acc: 0.7656
Epoch 7/200
200934/200934 [==============================] - 2s 11us/step - loss: 0.4630 - acc: 0.8039 - val_

Epoch 59/200
200934/200934 [==============================] - 2s 11us/step - loss: 0.1185 - acc: 0.9540 - val_loss: 0.0971 - val_acc: 0.9633
Epoch 60/200
200934/200934 [==============================] - 2s 11us/step - loss: 0.1151 - acc: 0.9554 - val_loss: 0.1181 - val_acc: 0.9520
Epoch 61/200
200934/200934 [==============================] - 2s 11us/step - loss: 0.1175 - acc: 0.9545 - val_loss: 0.1001 - val_acc: 0.9658
Epoch 62/200
200934/200934 [==============================] - 2s 11us/step - loss: 0.1141 - acc: 0.9563 - val_loss: 0.0931 - val_acc: 0.9688
Epoch 63/200
200934/200934 [==============================] - 2s 10us/step - loss: 0.1151 - acc: 0.9558 - val_loss: 0.1218 - val_acc: 0.9519
Epoch 64/200
200934/200934 [==============================] - 2s 11us/step - loss: 0.1116 - acc: 0.9569 - val_loss: 0.1107 - val_acc: 0.9618
Epoch 65/200
200934/200934 [==============================] - 2s 11us/step - loss: 0.1110 - acc: 0.9571 - val_loss: 0.1068 - val_acc: 0.9595
Epoch 66/200


200934/200934 [==============================] - 2s 8us/step - loss: 0.0855 - acc: 0.9674 - val_loss: 0.0861 - val_acc: 0.9700
Epoch 118/200
200934/200934 [==============================] - 2s 10us/step - loss: 0.0836 - acc: 0.9683 - val_loss: 0.1050 - val_acc: 0.9618
Epoch 119/200
200934/200934 [==============================] - 2s 9us/step - loss: 0.0804 - acc: 0.9701 - val_loss: 0.0687 - val_acc: 0.9711
Epoch 120/200
200934/200934 [==============================] - 2s 8us/step - loss: 0.0881 - acc: 0.9668 - val_loss: 0.0655 - val_acc: 0.9765
Epoch 121/200
200934/200934 [==============================] - 2s 8us/step - loss: 0.0828 - acc: 0.9687 - val_loss: 0.0654 - val_acc: 0.9765
Epoch 122/200
200934/200934 [==============================] - 2s 9us/step - loss: 0.0794 - acc: 0.9696 - val_loss: 0.0685 - val_acc: 0.9732
Epoch 123/200
200934/200934 [==============================] - 2s 8us/step - loss: 0.0799 - acc: 0.9696 - val_loss: 0.0834 - val_acc: 0.9702
Epoch 124/200
200934/20093

200934/200934 [==============================] - 2s 10us/step - loss: 0.0748 - acc: 0.9714 - val_loss: 0.0797 - val_acc: 0.9611
Epoch 176/200
200934/200934 [==============================] - 2s 11us/step - loss: 0.0679 - acc: 0.9742 - val_loss: 0.0691 - val_acc: 0.9705
Epoch 177/200
200934/200934 [==============================] - 2s 11us/step - loss: 0.0759 - acc: 0.9704 - val_loss: 0.0657 - val_acc: 0.9700
Epoch 178/200
200934/200934 [==============================] - 2s 11us/step - loss: 0.0698 - acc: 0.9733 - val_loss: 0.0525 - val_acc: 0.9821
Epoch 179/200
200934/200934 [==============================] - 2s 11us/step - loss: 0.0669 - acc: 0.9743 - val_loss: 0.0556 - val_acc: 0.9816
Epoch 180/200
200934/200934 [==============================] - 2s 11us/step - loss: 0.0695 - acc: 0.9733 - val_loss: 0.0581 - val_acc: 0.9736
Epoch 181/200
200934/200934 [==============================] - 2s 10us/step - loss: 0.0713 - acc: 0.9733 - val_loss: 0.0479 - val_acc: 0.9803
Epoch 182/200
200934

In [453]:
test_loss,test_acc = model.evaluate(numpy_test_inputs,test_label_categ)
print('test accuracy=',test_acc)

26790/26790 [==============================] - 0s 19us/step
test accuracy= 0.9727510265024263


## To save the Model architecture and weight 

In [454]:
model.save('trained.h5')